In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('ASP_07-2019_09-2022_A.csv')

In [ ]:
df.head()

,Time,Timezone,GHI_AVG,DNI_AVG,WDIR_AVG,WSPD_AVG,TEMP_AVG,HUM_AVG,RAIN_SUM
0,2019-07-01 00:00:00,ACST,-1.251141,-1.681195,226.046751,0.270745,4.599429,49.373845,0.0
1,2019-07-01 01:00:00,ACST,-1.105491,-1.564169,231.856001,0.438133,3.880017,50.883139,0.0
2,2019-07-01 02:00:00,ACST,-1.110939,-1.588327,235.262640,0.391107,3.119183,53.990385,0.0
3,2019-07-01 03:00:00,ACST,-0.995637,-1.483157,246.829333,0.444790,3.146530,54.673470,0.0
4,2019-07-01 04:00:00,ACST,-1.001849,-1.464027,249.490834,0.454146,5.106388,48.845702,0.0


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from datetime import datetime

In [ ]:
train_dates = pd.to_datetime(df['Time'])

In [ ]:
df.drop(df[df['GHI_AVG'] <= 0].index, inplace = True)

In [ ]:
df = df.drop(['RAIN_SUM'], axis=1)

In [ ]:
df = df.drop(['Timezone'], axis=1)

In [ ]:
df = df.drop(['DNI_AVG'], axis=1)

In [ ]:
df

,Time,GHI_AVG,WDIR_AVG,WSPD_AVG,TEMP_AVG,HUM_AVG
7,2019-07-01 07:00:00,13.691893,173.649389,1.563628,10.970904,35.616153
8,2019-07-01 08:00:00,205.014492,194.149584,1.109589,13.686888,28.857826
9,2019-07-01 09:00:00,384.967336,255.291556,1.292724,16.802849,24.344240
10,2019-07-01 10:00:00,566.681863,252.201000,1.562360,20.027784,29.721130
11,2019-07-01 11:00:00,670.365928,274.574500,2.092833,21.861736,31.835168
...,...,...,...,...,...,...
27806,2022-09-01 14:00:00,397.580556,98.092639,2.646124,12.566402,98.669837
27807,2022-09-01 15:00:00,120.298611,143.875417,1.976757,12.903492,96.680905
27808,2022-09-01 16:00:00,99.322222,141.260417,1.864640,12.915450,95.157149
27809,2022-09-01 17:00:00,28.101389,145.524306,1.942893,12.795519,95.269921


In [ ]:
df['GHI_AVG'].isnull().values.any()

True

In [ ]:
df['GHI_AVG'].interpolate(method="linear")

7         13.691893
8        205.014492
9        384.967336
10       566.681863
11       670.365928
            ...    
27806    397.580556
27807    120.298611
27808     99.322222
27809     28.101389
27810      0.883333
Name: GHI_AVG, Length: 16560, dtype: float64

In [ ]:
df['GHI_AVG'].isnull().values.any()

True

In [ ]:
df['GHI_AVG'].interpolate(option="spline")

7         13.691893
8        205.014492
9        384.967336
10       566.681863
11       670.365928
            ...    
27806    397.580556
27807    120.298611
27808     99.322222
27809     28.101389
27810      0.883333
Name: GHI_AVG, Length: 16560, dtype: float64

In [ ]:
df['GHI_AVG'].isnull().values.any()

True

In [ ]:
 df['GHI_AVG'].isnull().sum()

1972

In [ ]:
df.fillna(method='bfill', inplace=True)

In [ ]:
df['GHI_AVG'].isnull().sum()

0

In [ ]:
cols = list(df)[1:6]

In [ ]:
cols = [ 'GHI_AVG', 'WDIR_AVG', 'WSPD_AVG', 'TEMP_AVG','HUM_AVG' ]

In [ ]:
df_for_training = df[cols].astype(float)

In [ ]:
df_for_training

,GHI_AVG,WDIR_AVG,WSPD_AVG,TEMP_AVG,HUM_AVG
7,13.691893,173.649389,1.563628,10.970904,35.616153
8,205.014492,194.149584,1.109589,13.686888,28.857826
9,384.967336,255.291556,1.292724,16.802849,24.344240
10,566.681863,252.201000,1.562360,20.027784,29.721130
11,670.365928,274.574500,2.092833,21.861736,31.835168
...,...,...,...,...,...
27806,397.580556,98.092639,2.646124,12.566402,98.669837
27807,120.298611,143.875417,1.976757,12.903492,96.680905
27808,99.322222,141.260417,1.864640,12.915450,95.157149
27809,28.101389,145.524306,1.942893,12.795519,95.269921


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df_for_training_scaled=scaler.fit_transform(np.array(df_for_training).reshape(-1,1))

In [ ]:
#Empty lists to be populated using formatted training data
trainX = []
trainY = []

In [ ]:
n_future = 1 
n_past = 100

In [ ]:
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

In [ ]:
trainX, trainY = np.array(trainX), np.array(trainY)

In [ ]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (82700, 100, 1).
trainY shape == (82700, 1).


In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(trainY.shape[1]))

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 64)           16896     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit the model
history = model.fit(trainX, trainY, epochs=5, batch_size=16, validation_split=0.1, verbose=1)

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

Epoch 1/5
  83/4652 [..............................] - ETA: 10:38 - loss: 0.0442